# Imports and configuration.

In [1]:
%matplotlib inline

In [9]:
import lda
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import psycopg2
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text


import functions as f

In [3]:
matplotlib.style.use('ggplot')

# Load data

In [6]:
con = psycopg2.connect("host='localhost' dbname='explore' user='explore' password='Ln2bOYAVCG6utNUSaSZaIVMH'")
cursor = con.cursor()

cursor.execute('select category, count(*) from recipes_recipe group by category')

df = pd.read_sql_query('select * from recipes_recipe', con=con)
print('Loaded %s records' % df.shape[0])


Loaded 4506 records


In [798]:
# Extract features
vectorizer = CountVectorizer(
             stop_words='english'
           , ngram_range=(1, 2)
           , token_pattern='[A-Za-z]+'
           , min_df = 3
)

features = vectorizer.fit_transform(df.ingredient_txt)
## features is a document x term matrix.

wc = f.feature_counts(vectorizer, features)

wc.sort('count').tail(25).plot('word','count', kind='bar')

wc.sort('count').head(500).plot('word','count', kind='bar')

In [558]:
m = lda.LDA(n_topics=40, random_state=0, n_iter=100)
m.fit(features)
print('Finished running model')


Finished running model


# Evaluating the model.
## Convergence

In [559]:
plt.plot(m.loglikelihoods_, '-')
plt.title('Loglikelihood')


# Assessing topics

In [765]:
## Extracting topic data.

## most probable words by topic.
## TODO: check if these are properly sorted within each topic.
w = f.most_probable_words(m, vectorizer.get_feature_names(), 10)

## most probable documents by topic.
np.apply_along_axis(lambda i: df.iloc[i]['title'], 1, doc_ids)

doc_ids = np.argsort(m.doc_topic_, axis=0)[-4:-1,:].T

doc_probs = np.sort(m.doc_topic_, axis=0)[-4:-1,:].T


print('='*70)
for t in range(m.n_topics):
    print('topic: %s' % t)
    print('documents:')
    print pd.DataFrame([df.iloc[doc_ids[t,:]]['title'].values, doc_probs[t,:]]).T.sort(1, ascending=False).to_string(header=False, index=False)
#    print('\n'.join(df.iloc[doc_ids[t,:]]['title']))
    print('-----'.join(df.iloc[doc_ids[t,:]]['ingredient_txt']))
    print('-'*70)
    print w[w['label']==t][['word','prob']].sort('prob', ascending=False).T.to_string(index=False, header=False, float_format=lambda x: '% 4.3f' % x)
    print('='*70)




1 (2 1/2-to 3-pounds) head cauliflower
1/4 cup extra-virgin olive oil, divided
1 small garlic clove
1 to 2 tablespoons fresh lemon juice (to taste)
1/4 cup pitted Kalamata olives, finely chopped
-----
1 cup brine-cured pitted black olives (such as Kalamata)
1 cup brine-cured pitted green olives (such as Picholine)
1/4 cup extra-virgin olive oil
2 tablespoons chopped fresh thyme
2 tablespoons drained capers
1 tablespoon chopped fresh rosemary
3 garlic cloves, chopped
1 anchovy fillet packed in oil, drained
-----
6 tangerines, peeled
1/4 cup fresh tangerine juice
3 tablespoons sugar
1 tablespoon (or more) fresh lemon juice

----------------------------------------------------------------------
 lemon  juice  green  olives  extra  virgin   ripe   zest  beans
 0.083  0.082  0.049   0.047  0.043   0.037  0.036  0.023  0.020




2 teaspoons coriander seeds
1 cup (packed) dark brown sugar
1/2 teaspoons whole black peppercorns
Kosher salt
-----
1 cup Dijon mustard
1/2 cup mayonnaise 
1/2 cup honey
1/2 teaspoon kosher salt 
1/4 teaspoon cayenne pepper
1/4 teaspoon dry mustard powder
-----
2 1/4 pounds carrots with tops
2 tablespoons Sherry vinegar
1 tablespoon packed dark brown sugar
1 teaspoon cumin seeds, toasted
6 tablespoons extra-virgin olive oil
1/2 pound Savoy cabbage, thinly sliced (about 6 cups)

----------------------------------------------------------------------
 mustard  vinegar  kosher  dijon  cumin   wine  coriander  yellow  sugar
   0.114    0.069   0.060  0.052  0.032  0.030      0.030   0.025  0.024
topic: 37
documents:
               Tree-Trimming Trail Mix    0.50625
        Inside-Out Apple Pie à la Mode  0.4764706
 Mascerated Berries with Vanilla Cream  0.4555556

6 cups fresh mixed berries (scant 2 pounds), divided
3/4 cup sugar
1/4 cup fresh orange juice
-----
1 cup old-fashioned rolled


topic: 34
documents:
 Olive-Oil Toasts with Greens, Pine Nuts, and R...     0.5125
                                  Herbed Olive Oil  0.5083333
                               Italian Salsa Verde  0.5055556

1 garlic clove, minced
3/4 teaspoon (or more) fine sea salt
1/2 teaspoon (or more) black pepper
1/2 teaspoon dried crushed red pepper
1/2 teaspoon (packed) grated lemon peel
31/2 tablespoons fresh lemon juice
3/4 cup extra-virgin olive oil
3/4 cup (packed) chopped Italian parsley
1 tablespoon drained capers
2 anchovy fillets, minced
-----
3 cups (packed) fresh parsley leaves (about 1 bunch)
2 cups (packed) fresh mint leaves (about 2/3 of a bunch)
1 tablespoon finely grated lemon zest (from about 2 lemons)
1 1/4 cups extra-virgin olive oil
1/2 teaspoon kosher salt
-----
2 tablespoons extra-virgin olive oil, divided
1 baguette, cut into 24 slices (1/4-inch thick)
1/4 teaspoon kosher salt

----------------------------------------------------------------------
 extra  virgin  kosher  



1 1/2 cups bottled mild tomato chili sauce (such as Homade brand)
7 teaspoons fresh lemon juice 
2 teaspoons Worcestershire sauce 
1 teaspoon hot pepper sauce 
3 teaspoons (or more) prepared white cream-style horseradish
-----
1/2 cup mayonnaise (preferably Hellmann's or Best Foods)
1/3 cup ketchup (preferably Heinz)
2 tablespoons sweet relish
1 tablespoon finely chopped cornichons
1 tablespoon finely chopped red onion
1 teaspoon Worcestershire sauce
1/2 teaspoon garlic powder
1/2 teaspoon onion powder
1/4 teaspoon hot sauce (preferably Tabasco)
Kosher salt, freshly ground pepper
-----
2/3 cup mayonnaise
2 green onions, finely chopped
1 tablespoon hot chili sauce (such as sriracha)*

----------------------------------------------------------------------
   hot  paprika  smoked  green  worcestershire  prepared  tomato  mayonnaise  onion
 0.063    0.046   0.045  0.027           0.025     0.025   0.023       0.022  0.022
topic: 31
documents:
        Chicken Rice Casserole  0.5761905
  V


topic: 27
documents:
 Roasted New Potatoes with Red Onion, Garlic, a...  0.6230769
           Grilled Cheese and Fried Egg Sandwiches      0.605
                 Chard and Onion Omelet (Trouchia)       0.54

3 tablespoons olive oil
1 large red or white onion, quartered and thinly sliced crosswise
1 bunch chard, leaves only, chopped
Salt and freshly milled pepper
1 garlic clove
6 to 8 eggs, lightly beaten
2 tablespoons chopped parsley
2 tablespoons chopped basil
2 teaspoons chopped thyme
1 cup grated Gruyère
2 tablespoons freshly grated Parmesan
-----
12 slices pancetta (Italian bacon)
2 tablespoons butter
4 slices sourdough bread
8 thin slices provolone cheese
4 large eggs
1 small green onion, chopped
Parmesan cheese shavings
12 fresh basil leaves or arugula leaves
-----
4 ounces thinly sliced pancetta (Italian bacon), finely chopped
3 1/2 pounds 1 1/2-inch-diameter baby red potatoes
3 tablespoons olive oil
1 1/2 cups coarsely chopped red onion
8 garlic cloves, peeled
2 tablespoons ch

...


topic: 15
documents:
 Over-Easy Cheesy BLT       0.66
             Molletes  0.6107143
  Bacon Queso Fundido      0.605

4 slices  OSCAR MAYER Bacon
1  poblano chile, halved lengthwise, seeded and chopped
1 clove  garlic, minced
4 oz.  (1/2 of 8-oz. pkg.) PHILADELPHIA Cream Cheese, softened
1 pkg.  (8 oz.) KRAFT Shredded Pepper Jack Cheese
-----
1 tablespoon unsalted butter, softened
2 large rolls, preferably teleras, Portuguese, kaiser, or ciabatta, split
Generous 1/4 cup Refried Black Beans, homemade
 or canned
6 ounces Chihuahua cheese or provolone cheese, shredded 
Generous 1/2 cup Pico de Gallo with Lemon Zest
-----
2 slices OSCAR MAYER Bacon
1 egg
1 slice each Colby Jack and Mild Cheddar from KRAFT Colby Jack and Mild Cheddar Combo Pack Cheese Slices
1 piece French bread baguette (4 inch), partially split
1 Tbsp. KRAFT Mayo with Olive Oil Reduced Fat Mayonnaise
1 lettuce leaf
2 tomato slices

----------------------------------------------------------------------
 shredded  chedd


topic: 11
documents:
  Nectarine, Plum, and Raspberry Pie  0.7214286
 Italian Rice Pie (Pastiera di Riso)        0.7
                     Rosa's Biscuits    0.69375

2 1/2 cups all-purpose flour
3/4 cup sugar
2 1/2 teaspoons baking powder
1 1/2 teaspoons kosher salt, plus more
1 cup (2 sticks) chilled unsalted butter, cut into pieces; plus 6 tablespoons (3/4 stick), melted
3/4 cup chilled buttermilk
-----
3 1/4 cups all-purpose flour
6 tablespoons sugar
1 1/2 teaspoons baking powder
3/4 teaspoon salt
2 sticks (1 cup) cold unsalted butter, cut into tablespoon pieces
3 large eggs, lightly beaten
-----
2 1/2 cups all-purpose flour
1 1/2 sticks (3/4 cup) cold unsalted butter,  cut into 1/2" cubes 
1/4 cup cold vegetable shortening
1/2 teaspoon salt
5 to 7 tablespoons ice water

----------------------------------------------------------------------
 unsalted  flour  purpose  kosher  chilled  sticks   cold  sugar   eggs
    0.150  0.124    0.099   0.065    0.043   0.031  0.026  0.025  0.016


----------------------------------------------------------------------
 green  stems  cilantro  scallions  tender  vegetable   lime  separated  carrots
 0.052  0.048     0.041      0.034   0.033      0.029  0.025      0.025    0.022
topic: 8
documents:
     Monster Pops  0.7742857
   Sangria Blanco     0.7125
 Basic Cake Balls  0.6138889

18.25-ounce box cake mix 
9-by-13-inch cake pan 
Large mixing bowl
One 16-ounce container ready-made frosting 
Large metal spoon
Wax paper
2 baking sheets
Plastic wrap
32 ounces (2 pounds) candy coating 
Deep, microwave-safe plastic bowl 
Toothpicks 
Resealable plastic bag or squeeze bottle (optional)
-----
2 lemons, thinly sliced
1 pink grapefruit, thinly sliced
2 cups fresh pink grapefruit juice
1 375-ml bottle Dolin blanc or dry vermouth
1 1/2 cups Suze Saveur d'Autrefois
3/4 cup pisco
1/4 cup torn fresh mint leaves
2 750-ml bottles chilled Vinho Verde
-----
48 uncoated Basic Cake Balls
, chilled
64 ounces (4 pounds) purple candy coating
Deep, mic


topic: 4
documents:
 Cauliflower Soup with Chive Oil and Rye Crostini    0.50625
                                     Torta Mimosa  0.4555556
                             Charred-Bread Powder  0.3923077

Dried chiles de árbol or 1/2 teaspoon crushed red pepper flakes
4 ounces torn stale crusty country-style bread
-----
1 recipe Italian Sponge Cake
, made as 2 (9-inch cakes)
-----
2 1-ounce bunches chives
Kosher salt
3/4 cup plus 1 tablespoon olive oil
8 thin slices cut from Finnish rye bread (for crostini) or 4 slices pumpernickel bread, cut into 1/3" cubes (for croutons)
2 sprigs rosemary

----------------------------------------------------------------------
 kosher  style  country  store  bought   loaf  toasted    ham  homemade
  0.070  0.056    0.054  0.041   0.040  0.035    0.032  0.030     0.028
topic: 5
documents:
       Grilled Rib Eye with Paprika Vinaigrette  0.6066667
 Frisée and Radish Salad with Hazelnut Dressing  0.5611111
         Winter Lettuces with Pomegranate Seeds 

topic: 0
documents:
 Grilled Street Corn Salad with Cilantro Butter...  0.7214286
             Fresh Green Salsa (Salsa verde cruda)  0.7117647
                           Charred Tomatillo Salsa    0.69375

1/2 pound tomatillos, husked, rinsed, patted dry
1/2 large white onion, cut into 8 wedges
1 jalapeño or serrano chile, halved, seeded
2 tablespoons (or more) fresh lime juice
Kosher salt
-----
1/2 pound tomatillos (5 or 6), husked, rinsed, and coarsely chopped
1/2 cup coarsely chopped cilantro
2 fresh jalapeño or serrano chiles, coarsely chopped (including seeds), or more to taste
1 large garlic clove, peeled
2 tablespoons chopped white onion
1 1/2 teaspoons kosher salt
-----
1 ear of corn, shucked
4 tablespoons grated queso fresco
2 tablespoons chopped cilantro
Juice of 1/2 lime
1 teaspoon cayenne pepper
1/2 teaspoon ground cumin
2 tablespoons olive oil
Salt and freshly ground pepper to taste

----------------------------------------------------------------------
  lime  jalape    